In [115]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.insert(0, "../src/")
sys.path.insert(0, "../../../src/")
sys.path.insert(0, "../../../projects/pyskip_bench/src/")
sys.path.insert(0, "../../../projects/pyskip_blox/")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [140]:
import random
import numpy as np
import pyskip
import pyskip_3d
from pyskip_blox import minecraft, colors
from pyskip import functional as F
import ipywebrtc

SHOW_RUNS_MODE = False

In [142]:
def show_runs(t, runs_value=2):
    q = t.reshape(len(t))
    ret = pyskip.Tensor(len(t), val=q.to(bool).to(int)._tensor.array())
    run_points = runs_value * (
        (
            (q[:(len(q) - 1)] == q[1:])
        )
        & (q[1:] != 0)
    ).to(int)

    ret[1:] = ret[1:].max(run_points)
    ret[0:(len(ret) - 1)] = ret[0:(len(ret) - 1)].max(runs_value * (ret[1:] == 2).to(int))
    return ret.reshape(t.shape)
    

In [143]:
%%time
# level = minecraft.PySkipMinecraftLevel.load("../data/sample_minecraft_world.pickle.gz")
level = minecraft.PySkipMinecraftLevel.load("../data/Hogwarts1.0.pickle.gz")

CPU times: user 596 ms, sys: 0 ns, total: 596 ms
Wall time: 596 ms


In [144]:
%%time
t = level.megatensor()
# t = level.chunk_list[0].tensor
if SHOW_RUNS_MODE:
    t = show_runs(t)

CPU times: user 27.6 s, sys: 280 ms, total: 27.9 s
Wall time: 3.17 s


In [145]:
for i in range(20):
    print(f"{i}=>{F.sum((t == i).to(int))}")

0=>669083423
1=>1070920
2=>585086
3=>2185407
4=>2095
5=>26584
6=>0
7=>861692
8=>0
9=>7172236
10=>0
11=>0
12=>1429
13=>1122
14=>0
15=>0
16=>0
17=>28831
18=>327296
19=>0


In [146]:
config = pyskip_3d.VoxelConfig()
config[0] = pyskip_3d.EmptyVoxel()
if SHOW_RUNS_MODE:
    config[1] = pyskip_3d.ColorVoxel(255, 255, 255)
    config[2] = pyskip_3d.ColorVoxel(255, 105, 180)
else:    
    for i in range(1, 256):
        r, g, b = colors.color_for_id(i)
        config[i] = pyskip_3d.ColorVoxel(r, g, b)

In [147]:
%%time
mesh = pyskip_3d.generate_mesh(config, t._tensor)

CPU times: user 5.13 s, sys: 483 ms, total: 5.61 s
Wall time: 1.61 s


In [148]:
(len(mesh.triangles) * 4  + len(mesh.positions) * 4) / (level.num_runs() * 2 * 4) 

29.843810116358068

In [149]:
from pythreejs import *
import numpy as np
from IPython.display import display

In [150]:
%%time 

geometry = BufferGeometry(
    attributes={
        "position": BufferAttribute(
            np.array(mesh.positions, dtype='float32').reshape(-1, 3),
            normalized=False,
        ),
        "normal": BufferAttribute(
            np.array(mesh.normals, dtype='float32').reshape(-1, 3),
            normalized=False,
        ),
        "color": BufferAttribute(
            np.array(mesh.colors, dtype='uint8').reshape(-1, 4)
        ),
        "index": BufferAttribute(
            np.array(mesh.triangles, dtype='uint32'),
            normalized=False,
        ),
    },
)

CPU times: user 14.3 s, sys: 1.93 s, total: 16.2 s
Wall time: 15.9 s


In [151]:
js_mesh = Mesh(
    geometry=geometry,
    material=MeshLambertMaterial(vertexColors="VertexColors"),
    position=[0, 0, 0],
)

In [152]:
"""
camera_pos = level.xyz_to_pyskip_col(level.dense_dimensions())
camera = PerspectiveCamera(position=camera_pos, lookAt=(0,0,0), fov=20)
up = [0, 0, 0]
up[level.column_order[1]] = 1
camera.up = tuple(up)
"""

'\ncamera_pos = level.xyz_to_pyskip_col(level.dense_dimensions())\ncamera = PerspectiveCamera(position=camera_pos, lookAt=(0,0,0), fov=20)\nup = [0, 0, 0]\nup[level.column_order[1]] = 1\ncamera.up = tuple(up)\n'

In [153]:
light_position = level.xyz_to_pyskip_col((0, 256, 0))
point_light = PointLight(color='#ffffff', position=light_position)
global_light = AmbientLight(color='#333333')

#normals_helper = VertexNormalsHelper(js_mesh, size=0.5, color="#ffcc00")

scene = Scene(children=[js_mesh, camera, point_light, global_light], background="black")

In [160]:
renderer = Renderer(
    camera=camera, 
    background="#b0b0b0", 
    background_opacity=1,
    scene=scene,
   # controls=[OrbitControls(controlling=camera)],
    width=600,
    height=500,
)

display(renderer)

Renderer(background='#b0b0b0', camera=PerspectiveCamera(fov=20.0, position=(-514.5494735177291, 133.4029367061…

In [155]:
stream = ipywebrtc.WidgetStream(widget=renderer, max_fps=30)
recorder = ipywebrtc.ImageRecorder(filename='snapshot', format='png', stream=stream)
recorder

ImageRecorder(filename='snapshot', image=Image(value=b''), stream=WidgetStream(max_fps=30, widget=Renderer(bac…